In [3]:
import gradio as gr
import subprocess
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM

In [4]:
subprocess.run('pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True)

CompletedProcess(args='pip install flash-attn --no-build-isolation', returncode=1)

In [6]:
# Initialize Florence model
device = "cuda" if torch.cuda.is_available() else "cpu"
florence_model = AutoModelForCausalLM.from_pretrained('microsoft/Florence-2-base', trust_remote_code=True).to(device).eval()
florence_processor = AutoProcessor.from_pretrained('microsoft/Florence-2-base', trust_remote_code=True)


c:\AI_project\.venv\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-base:
- processing_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [7]:
def generate_caption(image):
    if not isinstance(image, Image.Image):
        image = Image.fromarray(image)
    
    inputs = florence_processor(text="<MORE_DETAILED_CAPTION>", images=image, return_tensors="pt").to(device)
    generated_ids = florence_model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=1024,
        early_stopping=False,
        do_sample=False,
        num_beams=3,
    )
    generated_text = florence_processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = florence_processor.post_process_generation(
        generated_text,
        task="<MORE_DETAILED_CAPTION>",
        image_size=(image.width, image.height)
    )
    prompt =  parsed_answer["<MORE_DETAILED_CAPTION>"]
    print("\n\nGeneration completed!:"+ prompt)
    return prompt

In [8]:
io = gr.Interface(generate_caption,
                  inputs=[gr.Image(label="Input Image")],
                  outputs = [gr.Textbox(label="Output Prompt", lines=2, show_copy_button = True),
                             # gr.Image(label="Output Image")
                            ]
                 )

In [12]:
io.launch(debug=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.




Generation completed!:The image is a grid of 16 square tiles arranged in a grid-like pattern. Each tile has a black background with small white dots scattered throughout. The dots are evenly spaced and appear to be made up of small squares of different sizes and shapes. The grid is arranged in two rows and three columns, with the top row being the largest and the bottom row being smaller. The image appears to be a black and white image.
Keyboard interruption in main thread... closing server.


In [13]:
io.close()

Closing server running on port: 7860
